In [1]:
#Author-Vishal Burman

## Concise Implementation of Multilayer Perceptron

In [18]:
from mxnet import gluon, init, autograd, nd
from mxnet.gluon import nn

## The Model

In [20]:
# Defining the model
# First layer is initialized from Gaussian distrbution with standard deviation of 0.01
# The hiddden layer is added with 256 nodes
# The output has 10 nodes because of 10 categories

In [4]:
net=nn.Sequential()
net.add(nn.Dense(256, activation='relu'), nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

In [21]:
# Defining the batch size of train_iterator and test_iterator

In [5]:
batch_size=256

In [22]:
# The function which loads the Fashion-MNIST into the memory

In [8]:
def load_data_fashion_mnist(batch_size, resize=None):
    """Load Fashion mnist into the memory"""
    dataset=gluon.data.vision
    trans=[dataset.transforms.Resize(resize)] if resize else []
    trans.append(dataset.transforms.ToTensor())
    trans=dataset.transforms.Compose(trans)
    mnist_train=dataset.FashionMNIST(train=True).transform_first(trans)
    mnist_test=dataset.FashionMNIST(train=False).transform_first(trans)
    return ((gluon.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)), (gluon.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)))

In [23]:
# Creating iterators for both training set and test-set

In [9]:
train_iter, test_iter=load_data_fashion_mnist(batch_size)

In [24]:
# The number of epochs set for training

In [10]:
num_epochs=10

In [ ]:
# We take the loss SoftMax Cross Entropy

In [11]:
loss=gluon.loss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':0.5})

In [12]:
import mxnet as mx

In [25]:
# The evaluation function which determines the accuracy of the model at any given time
# It takes two parameters-> data_iterator and the model

In [13]:
def evaluate_acc(data_iter, net):
    acc=mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iter):
        output=net(data)
        predictions=nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [14]:
#Writing the training loop

In [19]:
for epoch in range(1, num_epochs+1):
    cumulative_loss=0
    for X, y in train_iter:
        with autograd.record():
            y_hat=net(X)
            l=loss(y_hat,y)
        l.backward()
        trainer.step(batch_size)
        cumulative_loss+=nd.sum(l).asscalar()
    test_accuracy=evaluate_acc(test_iter, net)
    train_accuracy=evaluate_acc(train_iter, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (epoch, cumulative_loss/60000, train_accuracy, test_accuracy))
    

Epoch 1. Loss: 0.776368222173055, Train_acc 0.8108, Test_acc 0.8041
Epoch 2. Loss: 0.489579932975769, Train_acc 0.854, Test_acc 0.8502
Epoch 3. Loss: 0.4295759784062703, Train_acc 0.8511, Test_acc 0.8475
Epoch 4. Loss: 0.3949359818458557, Train_acc 0.8714666666666666, Test_acc 0.8669
Epoch 5. Loss: 0.36848887418111165, Train_acc 0.8783666666666666, Test_acc 0.872
Epoch 6. Loss: 0.35583555984497073, Train_acc 0.8771666666666667, Test_acc 0.8683
Epoch 7. Loss: 0.3412917427062988, Train_acc 0.8851, Test_acc 0.8786
Epoch 8. Loss: 0.3266287867863973, Train_acc 0.8898166666666667, Test_acc 0.878
Epoch 9. Loss: 0.31459743315378824, Train_acc 0.8884, Test_acc 0.8777
Epoch 10. Loss: 0.30656643781661985, Train_acc 0.8956, Test_acc 0.8812
